In [15]:
import pandas as pd
import random

In [16]:
word_counts = dict()
joints = {"neutral_pred": {}, "contradiction_pred": {}, "entailment_pred": {}, "neutral_actual": {}, "contradiction_actual": {}, "entailment_actual": {}}

In [17]:
def add_word(w, d):
    if w in d:
        d[w] += 1
    else:
        d[w] = 1

In [27]:
# filename = "method1_eval/train_preds"
filename = "report/method1/alpha_0.2_beta_0.1/sick/train_preds"
min_support = 20
min_conditional = 0.80
fracmin = "scaled"
fracmax = ""
preds = open(filename, 'r')
labels = open("../data\\sick\\cl_sick_train_lbl_file", 'r')
source = open("../data\\sick\\cl_sick_train_source_file", 'r')
# labels = open("../data/snli_1.0/cl_snli_train_lbl_file", 'r')
# source = open("../data/snli_1.0/cl_snli_train_source_file", 'r')

In [28]:
num_samples = 0

try:
    while True:
        pred = next(preds).replace("\n", "")
        label = next(labels).replace("\n", "")
        hypothesis = set(next(source).lower().split("|||")[1].replace("\n", "").split(" "))
        for word in hypothesis:
            add_word(word, word_counts)
            add_word(word, joints[pred + "_pred"])
            add_word(word, joints[label + "_actual"])
        num_samples += 1
        if num_samples % 10000 == 0:
            print(num_samples, "completed")
except StopIteration:
    pass

In [29]:
words = pd.DataFrame.from_dict(word_counts, orient='index').rename(columns={0: "count"})
words.head()

,count
boys,106
standing,530
yard,44
group,192
background,54


In [30]:
words = words.join(pd.DataFrame.from_dict(joints["neutral_pred"], orient='index').rename(columns={0: "neutral_pred"}))
words = words.join(pd.DataFrame.from_dict(joints["contradiction_pred"], orient='index').rename(columns={0: "contradiction_pred"}))
words = words.join(pd.DataFrame.from_dict(joints["entailment_pred"], orient='index').rename(columns={0: "entailment_pred"}))
words = words.join(pd.DataFrame.from_dict(joints["neutral_actual"], orient='index').rename(columns={0: "neutral_actual"}))
words = words.join(pd.DataFrame.from_dict(joints["contradiction_actual"], orient='index').rename(columns={0: "contradiction_actual"}))
words = words.join(pd.DataFrame.from_dict(joints["entailment_actual"], orient='index').rename(columns={0: "entailment_actual"}))
words.head()

,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual
boys,106,36.0,40.0,30.0,84.0,8.0,14.0
standing,530,118.0,236.0,176.0,356.0,60.0,114.0
yard,44,16.0,18.0,10.0,32.0,6.0,6.0
group,192,48.0,62.0,82.0,104.0,28.0,60.0
background,54,32.0,10.0,12.0,46.0,NaN,8.0


In [31]:
words = words.sort_values("count", ascending=False).reset_index().rename(columns={"index": "word"})
words.head()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual
0,is,7344,1588.0,2784.0,2972.0,4106.0,1136.0,2102.0
1,a,7230,1770.0,2586.0,2874.0,4230.0,924.0,2076.0
2,the,3614,838.0,1214.0,1562.0,1980.0,588.0,1046.0
3,man,2372,454.0,1012.0,906.0,1394.0,324.0,654.0
4,in,2058,650.0,680.0,728.0,1288.0,256.0,514.0


In [32]:
print(words.shape)
words = words[words["count"] > min_support]
words.shape

(1908, 8)


(389, 8)

In [33]:
words.head()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual
0,is,7344,1588.0,2784.0,2972.0,4106.0,1136.0,2102.0
1,a,7230,1770.0,2586.0,2874.0,4230.0,924.0,2076.0
2,the,3614,838.0,1214.0,1562.0,1980.0,588.0,1046.0
3,man,2372,454.0,1012.0,906.0,1394.0,324.0,654.0
4,in,2058,650.0,680.0,728.0,1288.0,256.0,514.0


In [34]:
words["neutral_pred"] = words["neutral_pred"] / words["count"]
words["contradiction_pred"] = words["contradiction_pred"] / words["count"]
words["entailment_pred"] = words["entailment_pred"] / words["count"]
words["neutral_actual"] = words["neutral_actual"] / words["count"]
words["contradiction_actual"] = words["contradiction_actual"] / words["count"]
words["entailment_actual"] = words["entailment_actual"] / words["count"]
words.head()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual
0,is,7344,0.216231,0.379085,0.404684,0.559096,0.154684,0.286220
1,a,7230,0.244813,0.357676,0.397510,0.585062,0.127801,0.287137
2,the,3614,0.231876,0.335916,0.432208,0.547869,0.162701,0.289430
3,man,2372,0.191400,0.426644,0.381956,0.587690,0.136594,0.275717
4,in,2058,0.315841,0.330418,0.353741,0.625850,0.124393,0.249757


In [35]:
words = words[(words["neutral_pred"] > min_conditional) | (words["contradiction_pred"] > min_conditional) | (words["entailment_pred"] > min_conditional)]
print(words.shape)
words.head()

(9, 8)


,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual
15,no,586,0.013652,0.965870,0.020478,0.406143,0.587031,0.006826
16,there,564,0.017730,0.971631,0.010638,0.411348,0.585106,0.003546
243,nobody,36,NaN,1.000000,NaN,0.333333,0.666667,NaN
301,tiger,28,NaN,0.928571,0.071429,0.642857,0.142857,0.214286
304,trunk,28,0.071429,0.071429,0.857143,0.214286,0.428571,0.357143


In [36]:
words = words.fillna(0)

In [200]:
words.loc[words["neutral_pred"] > min_conditional, "highest_pred"] = "neutral"
words.loc[words["contradiction_pred"] > min_conditional, "highest_pred"] = "contradiction"
words.loc[words["entailment_pred"] > min_conditional, "highest_pred"] = "entailment"
words.loc[(words["neutral_actual"] > words["contradiction_actual"]) & (words["neutral_actual"] > words["entailment_actual"]), "highest_label"] = "neutral"
words.loc[(words["contradiction_actual"] > words["neutral_actual"]) & (words["contradiction_actual"] > words["entailment_actual"]), "highest_label"] = "contradiction"
words.loc[(words["entailment_actual"] > words["contradiction_actual"]) & (words["entailment_actual"] > words["neutral_actual"]), "highest_label"] = "entailment"
words.head()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual,highest_pred,highest_label
196,nobody,2377,0.000421,0.999159,0.000421,0.004207,0.995372,0.000421,contradiction,contradiction
397,naked,1014,0.046351,0.904339,0.049310,0.064103,0.887574,0.048323,contradiction,contradiction
479,cats,818,0.035452,0.947433,0.017115,0.044010,0.936430,0.019560,contradiction,contradiction
524,favorite,734,0.920981,0.079019,0.000000,0.886921,0.102180,0.010899,neutral,neutral
551,winning,687,0.930131,0.061135,0.008734,0.896652,0.075691,0.027656,neutral,neutral


In [201]:
words.tail()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual,highest_pred,highest_label
5347,frown,22,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,contradiction,contradiction
5402,footwear,21,0.047619,0.047619,0.904762,0.142857,0.047619,0.809524,entailment,entailment
5434,disease,21,0.904762,0.047619,0.047619,0.857143,0.095238,0.047619,neutral,neutral
5464,videogames,21,0.047619,0.952381,0.000000,0.095238,0.904762,0.000000,contradiction,contradiction
5485,jealous,21,0.904762,0.000000,0.095238,0.857143,0.047619,0.095238,neutral,neutral


In [202]:
words["highest_pred"].value_counts()

neutral          80
contradiction    48
entailment        4
Name: highest_pred, dtype: int64

In [203]:
words[words["highest_pred"] != words["highest_label"]]

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual,highest_pred,highest_label


In [204]:
# def computefrac(minProb, minfrac, maxfrac, prob):
#     probrange = 1 - minProb
#     probperc = (prob - minProb) / probrange
#     fracrange = maxfrac - minfrac
#     return (fracrange * probperc) + minfrac

def computefrac(inProb, minfrac, maxfrac, prob):
    prob_reduce = prob - ((1 - prob) / 2)
    return prob_reduce / prob

In [205]:
labels = open("../data/snli_1.0/cl_snli_train_lbl_file", 'r')
source = open("../data/snli_1.0/cl_snli_train_source_file", 'r')
output = open("min%s,%s-%s-fixed.txt" % (min_conditional, fracmin, fracmax), 'w')

In [206]:
import random

num_samples = 0
dropped_words = 0

random.seed(42)

new_source = []

try:
    while True:
        label = next(labels).replace("\n", "")
        sample = next(source)
        sample = sample.split("|||")
        premise, hypothesis = sample[0], sample[1]
        new_source.append(premise + "|||")
        for word in hypothesis.split(" "):
            subset = words[(words["word"] == word.lower()) & (words["highest_label"] == label)]
            if subset.shape[0] and subset[label + "_actual"].iloc[0] > min_conditional:
                prob = computefrac(min_conditional, fracmin, fracmax, subset[label + "_actual"].iloc[0])
                num = random.random()
                if num > prob:
                    new_source.append(word + " ")
                else:
                    dropped_words += 1
            else:
                new_source.append(word + " ")
        new_source[-1] = new_source[-1][:-1]
        num_samples += 1
        # if num_samples == 3:
        #     print(repr("".join(new_source)))
        #     break
        if num_samples % 10000 == 0:
            print(num_samples, "completed")
            print(dropped_words, "dropped")
except StopIteration:
    pass

print(dropped_words, "dropped")
new_source = "".join(new_source)

10000 completed
157 dropped
20000 completed
317 dropped
30000 completed
477 dropped
40000 completed
638 dropped
50000 completed
788 dropped
60000 completed
966 dropped
70000 completed
1103 dropped
80000 completed
1254 dropped
90000 completed
1423 dropped
100000 completed
1565 dropped
110000 completed
1715 dropped
120000 completed
1888 dropped
130000 completed
2040 dropped
140000 completed
2188 dropped
150000 completed
2332 dropped
160000 completed
2490 dropped
170000 completed
2663 dropped
180000 completed
2806 dropped
190000 completed
2942 dropped
200000 completed
3131 dropped
210000 completed
3272 dropped
220000 completed
3443 dropped
230000 completed
3590 dropped
240000 completed
3744 dropped
250000 completed
3888 dropped
260000 completed
4030 dropped
270000 completed
4185 dropped
280000 completed
4331 dropped
290000 completed
4468 dropped
300000 completed
4621 dropped
310000 completed
4768 dropped
320000 completed
4919 dropped
330000 completed
5102 dropped
340000 completed
5279 dro

In [207]:
output.write(new_source)

60241580

In [208]:
word_counts = dict()
joints = {"neutral_new": {}, "contradiction_new": {}, "entailment_new": {}}

labels = open("../data/snli_1.0/cl_snli_train_lbl_file", 'r')
source = open("min%s,%s-%s-fixed.txt" % (min_conditional, fracmin, fracmax), 'r')

In [209]:
num_samples = 0

try:
    while True:
        pred = next(source).replace("\n", "")
        label = next(labels).replace("\n", "")
        hypothesis = set(next(source).lower().split("|||")[1].replace("\n", "").split(" "))
        for word in hypothesis:
            add_word(word, word_counts)
            add_word(word, joints[label + "_new"])
        num_samples += 1
        if num_samples % 10000 == 0:
            print(num_samples, "completed")
except StopIteration:
    pass

10000 completed
20000 completed
30000 completed
40000 completed
50000 completed
60000 completed
70000 completed
80000 completed
90000 completed
100000 completed
110000 completed
120000 completed
130000 completed
140000 completed
150000 completed
160000 completed
170000 completed
180000 completed
190000 completed
200000 completed
210000 completed
220000 completed
230000 completed
240000 completed
250000 completed
260000 completed
270000 completed


In [210]:
new_words = pd.DataFrame.from_dict(word_counts, orient='index').rename(columns={0: "count_new"})
new_words.head()

,count_new
at,20215
person,9974
an,11252
",",4178
a,164762


In [211]:
new_words = new_words.join(pd.DataFrame.from_dict(joints["neutral_new"], orient='index').rename(columns={0: "neutral_new"}))
new_words = new_words.join(pd.DataFrame.from_dict(joints["contradiction_new"], orient='index').rename(columns={0: "contradiction_new"}))
new_words = new_words.join(pd.DataFrame.from_dict(joints["entailment_new"], orient='index').rename(columns={0: "entailment_new"}))
new_words.head()

,count_new,neutral_new,contradiction_new,entailment_new
at,20215,6726.0,6685.0,6804.0
person,9974,3287.0,3402.0,3285.0
an,11252,3719.0,3784.0,3749.0
",",4178,1424.0,1353.0,1401.0
a,164762,54893.0,54917.0,54952.0


In [212]:
new_words = new_words.sort_values("count_new", ascending=False).reset_index().rename(columns={"index": "word"})
new_words.head()

,word,count_new,neutral_new,contradiction_new,entailment_new
0,.,219452,73123.0,73282.0,73047.0
1,a,164762,54893.0,54917.0,54952.0
2,the,121096,40266.0,40421.0,40409.0
3,is,110732,36886.0,36886.0,36960.0
4,are,62794,20894.0,20974.0,20926.0


In [213]:
new_words["neutral_new"] = new_words["neutral_new"] / new_words["count_new"]
new_words["contradiction_new"] = new_words["contradiction_new"] / new_words["count_new"]
new_words["entailment_new"] = new_words["entailment_new"] / new_words["count_new"]
new_words.head()

,word,count_new,neutral_new,contradiction_new,entailment_new
0,.,219452,0.333207,0.333932,0.332861
1,a,164762,0.333165,0.333311,0.333524
2,the,121096,0.332513,0.333793,0.333694
3,is,110732,0.333111,0.333111,0.333779
4,are,62794,0.332739,0.334013,0.333248


In [214]:
combined = words.merge(new_words, how='left', on="word")
combined.head()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual,highest_pred,highest_label,count_new,neutral_new,contradiction_new,entailment_new
0,nobody,2377,0.000421,0.999159,0.000421,0.004207,0.995372,0.000421,contradiction,contradiction,8.0,0.125000,0.750000,0.125000
1,naked,1014,0.046351,0.904339,0.049310,0.064103,0.887574,0.048323,contradiction,contradiction,504.0,0.327381,0.376984,0.295635
2,cats,818,0.035452,0.947433,0.017115,0.044010,0.936430,0.019560,contradiction,contradiction,73.0,0.273973,0.328767,0.397260
3,favorite,734,0.920981,0.079019,0.000000,0.886921,0.102180,0.010899,neutral,neutral,363.0,0.344353,0.319559,0.336088
4,winning,687,0.930131,0.061135,0.008734,0.896652,0.075691,0.027656,neutral,neutral,354.0,0.350282,0.290960,0.358757


In [215]:
combined.shape

(132, 14)

In [216]:
(combined["count"] - combined["count_new"]).sum()

12294.0